# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 07**: Structured Streaming with Files

**Date**: October 9th 2025

**Student Name**: Jaime Enrique Galindo Villegas

**Professor**: Pablo Camarillo Ramirez

# Create SparkSession

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on Structured Streaming (files)") \
    .master("spark://spark-master:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")
# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")

-------------------------------------------
Batch: 14
-------------------------------------------
+---------+-----+-------+----+
|timestamp|level|message|node|
+---------+-----+-------+----+
+---------+-----+-------+----+

-------------------------------------------
Batch: 15
-------------------------------------------
+---------+-----+-------+----+
|timestamp|level|message|node|
+---------+-----+-------+----+
+---------+-----+-------+----+

-------------------------------------------
Batch: 16
-------------------------------------------
+---------+-----+-------+----+
|timestamp|level|message|node|
+---------+-----+-------+----+
+---------+-----+-------+----+

-------------------------------------------
Batch: 17
-------------------------------------------
+---------+-----+-------+----+
|timestamp|level|message|node|
+---------+-----+-------+----+
+---------+-----+-------+----+

-------------------------------------------
Batch: 18
-------------------------------------------
+---------

# Create a data stream from a local script

### Connect Spark to the socket

In [8]:

!ls data/log_entries/
!pwd

log_f7b2d4a3-4839-49bf-9b50-e7e2b582a2ba.json
log_fc837e41-8ac5-42a5-a36d-d8aa03f60fa8.json


/opt/spark/work-dir


In [ ]:
from pcamarillor.spark_utils import SparkUtils

logs_schema = SparkUtils.generate_schema([
	("timestamp", "string"), 
	("level", "string"), 
	("message", "string"), 
	("node", "string")   
   ])

jsons_path = "/opt/spark/work-dir/data/log_entries/"
logs_df = spark.readStream \
            .format("json") \
            .schema(logs_schema) \
            .load(jsons_path)

error_df = logs_df.filter(logs_df.level == "ERROR")

query = error_df.writeStream \
            .outputMode("append") \
            .format("console") \
            .option("truncate", "false") \
            .start()

query.awaitTermination(30) # Wait 30 seconds before closing the stream

-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------------+-----+-------------------------+-------------+
|timestamp                 |level|message                  |node         |
+--------------------------+-----+-------------------------+-------------+
|2025-10-09T22:10:58.625169|ERROR|500 Internal Server Error|server-node-2|
+--------------------------+-----+-------------------------+-------------+

-------------------------------------------
Batch: 19
-------------------------------------------
+---------+-----+-------+----+
|timestamp|level|message|node|
+---------+-----+-------+----+
+---------+-----+-------+----+

-------------------------------------------
Batch: 1
-------------------------------------------
+---------+-----+-------+----+
|timestamp|level|message|node|
+---------+-----+-------+----+
+---------+-----+-------+----+

-------------------------------------------
Batch: 2
-----------------------------

False

-------------------------------------------
Batch: 16
-------------------------------------------
+--------------------------+-----+-------------------------+-------------+
|timestamp                 |level|message                  |node         |
+--------------------------+-----+-------------------------+-------------+
|2025-10-09T22:11:30.648004|ERROR|500 Internal Server Error|api-gateway-a|
+--------------------------+-----+-------------------------+-------------+

-------------------------------------------
Batch: 34
-------------------------------------------
+--------------------------+-----+-------------------------+-------------+
|timestamp                 |level|message                  |node         |
+--------------------------+-----+-------------------------+-------------+
|2025-10-09T22:11:30.648004|ERROR|500 Internal Server Error|api-gateway-a|
+--------------------------+-----+-------------------------+-------------+

-------------------------------------------
Batch: 1

In [28]:
sc.stop()

25/10/10 04:01:06 ERROR MicroBatchExecution: Query [id = a025fddc-0ae9-4518-a93f-993f35beb676, runId = 5d86be3f-f242-48b1-9a43-0124a587e5f8] terminated with error
org.apache.spark.SparkException: [INTERNAL_ERROR] The Spark SQL phase optimization failed with an internal error. You hit a bug in Spark or the Spark plugins you use. Please, report this bug to the corresponding communities or vendors, and provide the full stack trace. SQLSTATE: XX000
	at org.apache.spark.SparkException$.internalError(SparkException.scala:107)
	at org.apache.spark.sql.execution.QueryExecution$.toInternalError(QueryExecution.scala:643)
	at org.apache.spark.sql.execution.QueryExecution$.withInternalError(QueryExecution.scala:656)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$executePhase$1(QueryExecution.scala:278)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:804)
	at org.apache.spark.sql.execution.QueryExecution.executePhase(QueryExecution.scala:277)
	at org.apache.spark.sql